https://www.youtube.com/watch?v=oWZQb8nRGfk&ab_channel=ArturSpirin

In [1]:
class Table:
    
    def __init__(self, driver):
        self.driver = driver
        
    def get_column_info(self):
        
        column_info = []
        columns = self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[3]/thead/tr/th')
        for column in columns:
            column_info.append(str(column.text))
        return column_info
        
    def get_results(self, index=None):
        columns = self.get_column_info()
        data = {}
        elements = self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[3]/tbody/tr{}'
                                                      .format('[{}]'.format(index) if index else ''))
        
        for element in elements:
            current_index = elements.index(element) + 1 if not index else index #xpath counter starts at 1
            parsed_data = {}
            for column in columns:
                value = element.find_element_by_xpath('/html/body/div[2]/div[3]/div/table[3]/tbody/tr[{}]/td[{}]'
                                                      .format(current_index, columns.index(column)+1)).text
                parsed_data.update({column: str(value)})
            
            data.update({current_index: parsed_data})
            
        return data
    
    def get_number_of_results(self):
        
        return len(self.driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div/table[3]/tbody/tr'))
    
    def click(self, column, value):
        
        pass

In [2]:
import pandas as pd
import numpy as np

from selenium import webdriver
import time

In [3]:
start_year = 2000
end_year = 2021

driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')

for year in range(start_year, end_year):
    driver.get('https://basketball.realgm.com/nba/draft/past_drafts/{}'.format(str(year)))

    table = Table(driver)

    if year == start_year:
        df = pd.DataFrame(table.get_results()).T
        df['draft_year'] = year
        df = df.reset_index(drop=True)
    else:
        df_temp = pd.DataFrame(table.get_results()).T
        df_temp['draft_year'] = year
        df = pd.concat([df, df_temp])
        df = df.reset_index(drop=True)

df
#     print(table.get_column_info())
#     print(table.get_results())
#     print(table.get_number_of_results())

,Player,Pos,HT,WT,Age,YOS,Pre-Draft Team,Class,Nationality,draft_year
0,Malik Allen,F,6-10,255,22,11,Villanova,Sr,United States,2000
1,Eddie Gill,PG,6-0,190,21,8,Weber State,Sr,United States,2000
2,Ime Udoka,G-F,6-5,220,22,8,Portland State,Sr,United States / Nigeria,2000
3,Richie Frahm,G,6-5,210,22,5,Gonzaga,Sr,United States,2000
4,Pepe Sanchez,G,6-4,203,23,4,Temple,Sr,Argentina,2000
...,...,...,...,...,...,...,...,...,...,...
1169,Zavier Simpson,PG,6-0,190,23,0,Michigan,Sr,United States,2020
1170,Dwayne Sutton,SG,6-5,220,23,0,Louisville,Sr,United States,2020
1171,Marlon Taylor,G,6-5,209,23,0,LSU,Sr,United States,2020
1172,Tres Tinkle,SF,6-7,225,24,0,Oregon State,Sr,United States,2020


In [4]:
df.to_csv('C:/Users/17178/Documents/DS Projects/FanDuel/NBA Draft Project/Data/nba_draft_undrafted.csv')